<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e83f4682b8bae4e10a8d8abc5037b4a231eef86b3646ac4022b6ba02b4377eb6
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-16 16:25:42
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: -78.26 K (-0.55%)
Current PnL: -26.75 L (-17.28%)
CY Booked + Current PnL: -12.46 L (-8.05%)
-------------------
Total profit:  1.38 L
Total loss:  -28.13 L
-------------------
Total Booked + Current PnL: 14.52 L (11.36%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.19%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.39 L (65.14%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.18%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.96,-18.08,22.18,0.09,18311.0,-18221.0,82556.0,91.68,35.0,M-SC,2.33,253.0,-1.00,0.59,8.46,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.39,9.08,9.73,19.69,20350.0,17403.0,209145.0,-1.63,58.0,X-MC,2.53,78.0,0.86,1.50,16.49,XY25,NTT,AC
39,INDIGOPNTS,1408.00,-1.95,-13.40,15.50,0.02,23431.0,-23391.0,151168.0,130.47,49.0,M-SC,14.78,234.0,-1.00,1.08,31.49,OX40N,NTT,PAINTS
50,MASFIN,398.61,-2.76,-2.79,25.55,22.05,24336.0,-2730.0,95250.0,-15.99,52.0,H-SC,7.21,164.0,-0.11,0.68,38.45,XR,ATH,FINANCE
43,ITC,452.00,-0.15,-1.55,12.52,10.78,29622.0,-3717.0,236601.0,-41.15,45.0,X-LC,1.96,5.0,-0.13,1.69,3.61,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,CERA,9475.00,3.34,-30.01,77.02,23.89,94818.0,-52795.0,123108.0,-32.46,41.0,H-SC,10.56,157.0,-0.56,0.88,6.32,OX40N,NTT,CERAMICS
25,FINCABLES,1641.55,2.82,-1.26,110.56,107.91,161180.0,-1859.0,145785.0,-19.74,60.0,M-SC,12.52,216.0,-0.01,1.04,8.70,OX40N,ATH,CABLES
6,ASIANTILES,137.00,2.05,-16.81,118.40,81.67,92395.0,-15774.0,78036.0,7011.11,46.0,L-SC,12.37,271.0,-0.17,0.56,51.12,XR,NTT,CERAMICS
44,JCHAC,2282.00,1.55,-33.93,51.35,-0.01,43358.0,-43368.0,84437.0,17182.61,39.0,M-SC,7.33,233.0,-1.00,0.60,8.07,OX40N,NTT,AC
61,ROUTE,2227.21,1.41,-47.21,220.21,69.05,154700.0,-62815.0,70251.0,-57.43,52.0,H-SC,22.89,140.0,-0.41,0.50,8.34,SR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,-4.03,-40.78,128.44,35.27,86360.0,-46311.0,67238.0,-53.72,47.0,L-SC,10.75,270.0,-0.54,0.48,64.57,XR,NTT,HOTELS
57,RAJOOENG,143.10,-2.86,-35.72,117.18,39.61,77210.0,-36610.0,65890.0,-56.61,34.0,H-SC,28.73,135.0,-0.47,0.47,3.81,AR,ATH,MISC
30,HATHWAY,31.40,-2.78,-30.57,149.60,73.29,112918.0,-33240.0,75480.0,776.81,40.0,H-SC,8.06,172.0,-0.29,0.54,1.78,XR,NTT,ENTERTAINMENT
50,MASFIN,398.61,-2.76,-2.79,25.55,22.05,24336.0,-2730.0,95250.0,-15.99,52.0,H-SC,7.21,164.0,-0.11,0.68,38.45,XR,ATH,FINANCE
21,DEN,75.00,-2.41,-40.86,140.23,42.07,70048.0,-34512.0,49952.0,78.76,46.0,M-SC,7.82,235.0,-0.49,0.36,6.08,AR,NTT,ENTERTAINMENT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,2.82,-1.26,110.56,107.91,161180.0,-1859.0,145785.0,-19.74,60.0,M-SC,12.52,216.0,-0.01,1.04,8.70,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.95,-13.40,15.50,0.02,23431.0,-23391.0,151168.0,130.47,49.0,M-SC,14.78,234.0,-1.00,1.08,31.49,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.96,-18.08,22.18,0.09,18311.0,-18221.0,82556.0,91.68,35.0,M-SC,2.33,253.0,-1.00,0.59,8.46,OX40N,NTT,DURABLES
66,SIS,528.00,0.06,-24.18,59.93,21.26,50454.0,-26844.0,84188.0,1986.01,50.0,H-SC,5.12,166.0,-0.53,0.60,13.75,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.82,-25.07,51.43,13.47,103924.0,-67599.0,202068.0,-69.28,35.0,H-SC,5.21,158.0,-0.65,1.45,2.75,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.96,6.32,62.02,72.27,111734.0,10710.0,180158.0,-8.8,63.0,M-LC,2.45,99.0,0.1,1.29,12.89,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.96,6.32,62.02,72.27,111734.0,10710.0,180158.0,-8.80,63.0,M-LC,2.45,99.0,0.10,1.29,12.89,XR,NTT,IT
25,FINCABLES,1641.55,2.82,-1.26,110.56,107.91,161180.0,-1859.0,145785.0,-19.74,60.0,M-SC,12.52,216.0,-0.01,1.04,8.70,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.99,-4.54,116.56,106.72,137231.0,-5602.0,117734.0,-52.73,33.0,H-SC,3.01,139.0,-0.04,0.84,17.77,AR,ATH,MISC
1,ABB,7934.00,-0.81,-3.82,51.35,45.57,129205.0,-10003.0,251616.0,-38.17,56.0,H-MC,4.55,120.0,-0.08,1.80,7.06,AR,NTT,ELECTRICAL
37,IEX,219.00,-1.35,-4.72,55.98,48.63,106655.0,-9431.0,190523.0,-36.71,46.0,H-SC,14.07,136.0,-0.09,1.36,6.29,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.37,-18.25,90.17,55.47,179345.0,-44389.0,198897.0,-27.60,25.0,X-MC,12.93,64.0,-0.25,1.42,1.91,X40N,ATH,FINANCE
53,PAGEIND,51605.08,-1.05,-9.75,41.95,28.11,76255.0,-19635.0,181775.0,-34.62,28.0,X-MC,12.94,55.0,-0.26,1.30,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-0.49,-29.20,73.69,22.98,117862.0,-65955.0,159943.0,-26.74,29.0,X-LC,6.80,3.0,-0.56,1.15,0.60,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-1.08,-38.79,120.89,35.20,95212.0,-49911.0,78759.0,3.25,31.0,X-SC,16.12,92.0,-0.52,0.56,0.04,X40,NTT,FOOTWEAR
8,AWL,485.00,-1.32,-23.06,99.51,53.50,231022.0,-69591.0,232160.0,-62.74,31.0,X-MC,2.80,58.0,-0.30,1.66,3.12,XY24,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.19,-1.23,26.03,24.48,64751.0,-3102.0,248754.0,-81.09,44.0,X-SC,0.33,86.0,-0.05,1.78,21.62,X40N,NTT,MISC
43,ITC,452.0,-0.15,-1.55,12.52,10.78,29622.0,-3717.0,236601.0,-41.15,45.0,X-LC,1.96,5.0,-0.13,1.69,3.61,X40,NTT,FMCG
33,HINDUNILVR,2922.0,-0.54,-11.26,28.10,13.67,64099.0,-28953.0,228110.0,-17.95,32.0,X-LC,2.15,14.0,-0.45,1.63,7.69,XY25,NTT,FMCG
36,ICICIPRULI,790.0,-1.63,5.57,23.83,30.73,49560.0,10967.0,207972.0,-19.08,58.0,X-MC,2.24,75.0,0.22,1.49,19.04,X40,ATH,INSURANCE
78,UNITDSPR,1644.0,0.62,6.62,13.27,20.77,33127.0,15497.0,249641.0,-2.63,58.0,X-MC,2.46,62.0,0.47,1.79,13.99,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.35,-25.56,120.50,64.15,213454.0,-60811.0,177140.0,-56.31,34.0,X-SC,2.60,82.0,-0.28,1.27,0.00,XY24,BTT,CEMENT
53,PAGEIND,51605.08,-1.05,-9.75,41.95,28.11,76255.0,-19635.0,181775.0,-34.62,28.0,X-MC,12.94,55.0,-0.26,1.30,0.00,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-1.08,-38.79,120.89,35.20,95212.0,-49911.0,78759.0,3.25,31.0,X-SC,16.12,92.0,-0.52,0.56,0.04,X40,NTT,FOOTWEAR
75,TMPV,600.00,-0.49,-29.20,73.69,22.98,117862.0,-65955.0,159943.0,-26.74,29.0,X-LC,6.80,3.0,-0.56,1.15,0.60,XY24,NTT,AUTO
59,RELAXO,1176.00,-0.80,-48.07,193.27,52.31,145702.0,-69772.0,75388.0,-44.73,40.0,X-SC,6.95,91.0,-0.48,0.54,0.91,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.55,-29.38,102.27,42.84,46949.0,-19099.0,45907.0,-53.72,43.0,X-SC,35.52,83.0,-0.41,0.33,3.86,XY24,NTT,MISC
59,RELAXO,1176.00,-0.80,-48.07,193.27,52.31,145702.0,-69772.0,75388.0,-44.73,40.0,X-SC,6.95,91.0,-0.48,0.54,0.91,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.08,-38.79,120.89,35.20,95212.0,-49911.0,78759.0,3.25,31.0,X-SC,16.12,92.0,-0.52,0.56,0.04,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.72,-4.82,30.36,24.08,36684.0,-6121.0,120829.0,-12.55,32.0,X-SC,5.12,89.0,-0.17,0.87,13.85,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.16,-16.47,71.59,43.32,97391.0,-26832.0,136040.0,-28.18,37.0,X-SC,4.74,90.0,-0.28,0.97,4.45,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.78,-11.87,36.97,20.71,123232.0,-44907.0,333330.0,-23.95,59.0,X-LC,4.46,1.0,-0.36,2.39,11.38,X40,ATH,IT
41,INFY,1972.00,-0.86,8.44,23.80,34.24,80751.0,26400.0,339288.0,-15.65,59.0,X-LC,4.01,2.0,0.33,2.43,17.42,X40,NTT,IT
75,TMPV,600.00,-0.49,-29.20,73.69,22.98,117862.0,-65955.0,159943.0,-26.74,29.0,X-LC,6.80,3.0,-0.56,1.15,0.60,XY24,NTT,AUTO
81,VBL,671.64,-0.09,-3.62,40.76,35.67,124082.0,-11420.0,304422.0,-15.14,56.0,X-LC,4.13,4.0,-0.09,2.18,9.74,X40N,ATH,FMCG
43,ITC,452.00,-0.15,-1.55,12.52,10.78,29622.0,-3717.0,236601.0,-41.15,45.0,X-LC,1.96,5.0,-0.13,1.69,3.61,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.73,-37.61,113.01,32.89,54435.0,-29038.0,48168.0,-695.06,54.0,L-MC,5.97,259.0,-0.53,0.34,32.65,XR,NTT,BANKS
6,ASIANTILES,137.00,2.05,-16.81,118.40,81.67,92395.0,-15774.0,78036.0,7011.11,46.0,L-SC,12.37,271.0,-0.17,0.56,51.12,XR,NTT,CERAMICS
50,MASFIN,398.61,-2.76,-2.79,25.55,22.05,24336.0,-2730.0,95250.0,-15.99,52.0,H-SC,7.21,164.0,-0.11,0.68,38.45,XR,ATH,FINANCE
13,BSOFT,831.70,-1.99,-22.75,95.19,50.79,103430.0,-31997.0,108656.0,-1.04,62.0,H-SC,6.32,151.0,-0.31,0.78,26.51,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.95,-13.40,15.50,0.02,23431.0,-23391.0,151168.0,130.47,49.0,M-SC,14.78,234.0,-1.00,1.08,31.49,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.7,-1.99,-22.75,95.19,50.79,103430.0,-31997.0,108656.0,-1.04,62.0,H-SC,6.32,151.0,-0.31,0.78,26.51,XR,ATH,IT
84,WIPRO,420.0,-0.96,6.32,62.02,72.27,111734.0,10710.0,180158.0,-8.80,63.0,M-LC,2.45,99.0,0.10,1.29,12.89,XR,NTT,IT
41,INFY,1972.0,-0.86,8.44,23.80,34.24,80751.0,26400.0,339288.0,-15.65,59.0,X-LC,4.01,2.0,0.33,2.43,17.42,X40,NTT,IT
1,ABB,7934.0,-0.81,-3.82,51.35,45.57,129205.0,-10003.0,251616.0,-38.17,56.0,H-MC,4.55,120.0,-0.08,1.80,7.06,AR,NTT,ELECTRICAL
27,GLAXO,3466.2,0.29,3.79,34.06,39.15,70450.0,7556.0,206840.0,-16.56,53.0,X-MC,10.70,60.0,0.11,1.48,33.89,X40,ATH,PHARMA


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.27
1,25,44.88
2,50,76.68


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.90
MC    30.99
LC    24.08
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.72
X40      22.96
X40N     13.09
XR        9.73
XY25      9.09
AR        9.05
OX40N     7.57
SR        0.99
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.51
H-SC    23.76
X-LC    20.65
M-SC    11.68
X-SC     8.05
H-MC     4.82
L-SC     1.41
M-MC     1.32
M-LC     1.29
H-LC     1.13
L-LC     1.01
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.39,-7.39,41.92
IT,13.01,-15.78,73.99
FINANCE,9.45,-20.63,73.09
MISC,7.25,-28.17,80.60
ELECTRICAL,6.02,-11.44,52.25
PAINTS,5.86,-12.94,29.71
INSURANCE,4.77,-1.86,36.72
PHARMA,4.12,-2.08,34.21
AUTO,2.82,-33.58,79.89


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3194870.0,21
AR,1308449.0,10
XR,1301504.0,13
X40,1014369.0,14
X40N,857417.0,9
OX40N,731447.0,10
XY25,372034.0,6
SR,281864.0,2
MH,77014.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3658594.0,25
M-SC,1427510.0,15
X-MC,1403511.0,16
X-LC,843891.0,11
X-SC,794284.0,8
H-MC,406306.0,3
L-SC,264163.0,3
M-LC,111734.0,1
H-LC,75510.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1279755.0      6
           AR         895232.0      5
M-SC       XY24       823280.0      6
H-SC       XR         782807.0      7
X-MC       X40        490066.0      7
           XY24       403030.0      3
H-SC       OX40N      341922.0      4
X-LC       X40        334950.0      5
X-MC       X40N       333414.0      4
X-SC       X40N       307844.0      3
M-SC       OX40N      304117.0      5
X-SC       XY24       297087.0      3
H-SC       SR         281864.0      2
X-LC       X40N       216159.0      2
H-MC       AR         211367.0      2
X-LC       XY24       196779.0      2
H-MC       XY24       194939.0      1
X-SC       X40        189353.0      2
L-SC       XR         178755.0      2
X-MC       XY25       177001.0      2
M-SC       XR         173773.0      2
           AR         126340.0      2
M-LC       XR         111734.0      1
X-LC       XY25        96003.0      2
L-SC       OX40N       85408.0      1
H-SC       MH          77014.0      1
H-LC       AR          75510.0      1
M-MC       XY25        60838.0      1
L-MC       XR          54435.0      1
L-LC       XY25        38192.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
